In [2]:
import pandas as pd
import numpy as np
from pandasql import sqldf as sql

In [3]:
df = pd.read_csv('../data/final.csv')
df_cops = pd.read_csv('../data/2010_police_num_by_county.csv')
df_people = pd.read_csv('../data/people.csv')

In [4]:
pd.set_option('display.max_columns', 35)

In [5]:
df.shape

(3150, 23)

In [6]:
df_cops.shape

(3034, 4)

In [7]:
df_people.shape

(3212, 7)

In [8]:
df = pd.merge(df, df_cops, how='left', left_on='foreign_key', right_on='foreign_police')
df.shape

(3342, 27)

In [9]:
df = df.dropna()

In [10]:
df = pd.merge(df, df_people, how='left', left_on='fips', right_on='FIPS_people')
df.shape

(2705, 34)

In [11]:
df = df.dropna()

In [12]:
df.shape

(2660, 34)

In [13]:
df.head(2)

,state,fips,county_name,unemployment_rate_2010,population_total_2010,urban_population_prc,rural_population_prc,crime_per_capita,per_capita_sme_num,per_capita_large_num,avg_ann_pay_per_emp_sme,avg_ann_pay_per_emp_large,avg_ann_pay_per_emp_total,population_jail_prc,2008_dem_%_vote,2008_rep_%_vote,2008_other_%_vote,smoke_percent_2010,popul_hs_grad_prc,popul_college_grad_prc,popul_single_paren_prc,liquor_stores_per10k,foreign_key,state_police,county_police,number_police,foreign_police,FIPS_people,WhiteNonHispanicPct2010,BlackNonHispanicPct2010,AsianNonHispanicPct2010,NativeAmericanNonHispanicPct2010,HispanicPct2010,MultipleRacePct2010
0,AL,1001.0,Autauga,8.8,"54,571",0.579978377,0.420021623,0.002620439,0.012259259,0.002565465,22667.03037,31792.20223,26856.69322,0.002913635,0.257730214,0.736136373,0.006133412,25.77,72.0,21.0,9.0,0.4,Alabama Autauga,Alabama,Autauga,64,Alabama Autauga,1001.0,77.25,17.58,0.86,0.40,2.40,1.59
1,AL,1003.0,Baldwin,9.9,"182,265",0.577209009,0.422790991,0.002112309,0.020108084,0.002019038,28664.84962,29390.96748,28943.22956,0.004030066,0.238119219,0.752594794,0.009285986,22.97,67.0,26.0,9.0,0.8,Alabama Baldwin,Alabama,Baldwin,271,Alabama Baldwin,1003.0,83.50,9.31,0.74,0.63,4.38,1.49


In [14]:
df = df.drop(columns=['fips', 'foreign_key', 'state_police', 'county_police', 'foreign_police', 'FIPS_people'])

In [15]:
j=[]
for i in df['population_total_2010']:
    j.append(i.replace(',', ''))
df['population_total_2010'] = j
df['population_total_2010']

0        54571
1       182265
2        22915
3        57322
4        20947
         ...  
2700     43806
2701     21294
2702     21118
2703      8533
2704      7208
Name: population_total_2010, Length: 2660, dtype: object

In [16]:
k=[]
for m in df['number_police']:
    k.append(m.replace(',', ''))
df['number_police'] = k

In [17]:
cols = ['avg_ann_pay_per_emp_large', 'avg_ann_pay_per_emp_sme', 'avg_ann_pay_per_emp_total']
for i in cols:
    df[i] = [txt if txt != '#DIV/0!' else np.nan for txt in df[i]]

In [18]:
col_names = list(df.columns)
col_names = [i for i in col_names if not i in ['state', 'county_name']]

for b in col_names:
    try:
        df[b] = df[b].astype(float)
    except:
        pass

In [19]:
df = df.dropna(subset=['avg_ann_pay_per_emp_sme', 'avg_ann_pay_per_emp_total'])

In [20]:
df.shape

(2619, 28)

In [21]:
df = df.fillna(0)

In [22]:
per_capita_police = df['number_police'] / df['population_total_2010']

In [23]:
police_1000 = per_capita_police * 1000

In [24]:
df = df.rename(columns={'number_police': 'police_per_1000'})

In [25]:
df['police_per_1000'] = police_1000

In [26]:
df.to_csv('../data/final_work_data.csv', index=False)